In [37]:
import pickle

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

data = pd.read_parquet('../data/data.parquet')

inputs = ["x", "y"]
outputs = ["f1", "f2"]
dfin = data[inputs]
dfout = data[outputs]
x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

scaled_lb = dfin.min()[inputs].to_numpy()
scaled_ub = dfin.max()[inputs].to_numpy()
dict_scalers = {"x_offset": x_offset, "x_factor": x_factor, "y_offset": y_offset, "y_factor": y_factor,"scaled_ub": scaled_ub, "scaled_lb": scaled_lb}
pickle.dump(dict_scalers,open("scalers.pkl","wb"))

In [36]:
# Créer le modèle PMC
model = Sequential([
    Dense(64, input_dim=2, activation='sigmoid'),  # Couche cachée de 64 neurones
    Dense(32, activation='sigmoid'),  # Couche cachée de 32 neurones
    Dense(2)  # Couche de sortie avec 2 neurones (pour f1 et f2)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
x = dfin.to_numpy()
y = dfout.to_numpy()

history = model.fit(x, y, epochs=100)
# Évaluer le modèle
#loss, mae = model.evaluate(X_test, y_test)
#print(f'Loss: {loss}, MAE: {mae}')

model.save("plane_nn_relu.keras")


Epoch 1/100


/Users/philippe.stepniewski/Library/Caches/pypoetry/virtualenvs/hybrid-models-article-KtAidD1n-py3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.0825 - mae: 0.8571
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8591 - mae: 0.7614
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7272 - mae: 0.7034
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5848 - mae: 0.6202
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4531 - mae: 0.5425
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4195 - mae: 0.5160
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3859 - mae: 0.4971
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3743 - mae: 0.4839
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3552 - mae: 0.4668
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3353 - mae: 0.4554
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3199 - mae: 0.4395
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3160 - mae: 0.4410
Epoch 13/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/st

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Générer une grille de valeurs pour x et y
x_values = np.linspace(X['x'].min(), X['x'].max(), 50)
y_values = np.linspace(X['y'].min(), X['y'].max(), 50)
X_grid, Y_grid = np.meshgrid(x_values, y_values)
grid_data = np.c_[X_grid.ravel(), Y_grid.ravel()]

# Prédire les valeurs de f1 et f2 pour la grille
predictions = model.predict(grid_data)
grid_data = grid_data
F1_grid = predictions[:, 0].reshape(X_grid.shape)
F2_grid = predictions[:, 1].reshape(X_grid.shape)

# Créer une figure avec deux sous-graphes pour les fonctions f1 et f2
fig = plt.figure(figsize=(14, 6))

# Plot 3D pour f1
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X_grid, Y_grid, F1_grid, cmap='viridis')
ax1.set_title('Fonction apprise pour F1')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('f1')

# Plot 3D pour f2
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X_grid, Y_grid, F2_grid, cmap='plasma')
ax2.set_title('Fonction apprise pour F2')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('f2')

# Afficher les plots
plt.show()
